In [1]:
# 📦 Imports
import pandas as pd
import torch
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
from collections import Counter

# 🔧 Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

RANDOM_SEED = 42
set_seed(RANDOM_SEED)

# 📁 Load data
df1 = pd.read_csv("/content/Train_data.csv")[['text', 'label']]
df2 = pd.read_csv("/content/medical_data.csv").rename(columns={"Patient_Problem": "text", "Disease": "label"})[['text', 'label']]
df_all = pd.concat([df1, df2]).reset_index(drop=True)

# ❗ Entferne Klassen mit weniger als 2 Beispielen (für stratified split notwendig)
counts = df_all['label'].value_counts()
labels_to_remove = counts[counts < 2].index.tolist()
df_all_filtered = df_all[~df_all['label'].isin(labels_to_remove)].reset_index(drop=True)

print(f"Original dataset size: {len(df_all)}")
print(f"Filtered dataset size (removed classes with < 2 samples): {len(df_all_filtered)}")
print(f"Removed labels: {labels_to_remove}")

# 🔠 Label encoding auf gefiltertem Datensatz
le = LabelEncoder()
df_all_filtered['label'] = le.fit_transform(df_all_filtered['label'])
num_labels = len(le.classes_)

# 🔤 Model und Tokenizer (PubMedBERT)
model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 📦 Dataset-Klasse
class MedicalDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# 🧪 Stratified Train/Test Split mit fixiertem Seed
X_train, X_test, y_train, y_test = train_test_split(
    df_all_filtered["text"].tolist(),
    df_all_filtered["label"].tolist(),
    test_size=0.2,
    random_state=RANDOM_SEED,
    stratify=df_all_filtered["label"].tolist()
)

print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")
print("Train label distribution:", Counter(y_train))
print("Test label distribution:", Counter(y_test))

# 🏗️ DataLoader
train_loader = DataLoader(MedicalDataset(X_train, y_train), batch_size=16, shuffle=True)
test_loader = DataLoader(MedicalDataset(X_test, y_test), batch_size=16)

# 🧠 Modell-Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# 🛑 Early Stopping Parameter
patience = 3
best_val_loss = float("inf")
patience_counter = 0

train_losses = []
val_losses = []

# 🔁 Trainingsschleife mit Early Stopping
epoch = 0
while True:
    epoch += 1
    print(f"\nEpoch {epoch}")
    model.train()
    total_train_loss = 0

    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            total_val_loss += outputs.loss.item()

    avg_val_loss = total_val_loss / len(test_loader)
    val_losses.append(avg_val_loss)

    print(f"Train Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"\nEarly stopping triggered at epoch {epoch}")
        break

# 📥 Bestes Modell laden
model.load_state_dict(best_model_state)
model.eval()

# 🧾 Evaluation
preds = []
true = []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = model(**batch).logits
        preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true.extend(batch["labels"].cpu().numpy())

used_labels = np.unique(true)
used_names = le.inverse_transform(used_labels)

print("\n✅ Classification Report:\n")
print(classification_report(true, preds, labels=used_labels, target_names=used_names))
print("✅ Accuracy:", accuracy_score(true, preds))
print("✅ Precision:", precision_score(true, preds, average="weighted"))
print("✅ Recall:", recall_score(true, preds, average="weighted"))
print("✅ F1 Score:", f1_score(true, preds, average="weighted"))


Original dataset size: 1607
Filtered dataset size (removed classes with < 2 samples): 1517
Removed labels: ['Chronic Obstructive Pulmonary Disease (COPD)', 'Benign Positional Vertigo', 'Pericarditis', 'Mononucleosis', 'Pancreatitis', 'Migraine with Aura', 'Esophageal Reflux', 'Diabetes Type 2', 'Lung Cancer', 'Scabies', 'Pulmonary Embolism', 'Gastrointestinal Infection', 'Myopia and Hyperopia', 'Seasonal Allergies', 'Cystitis', 'Gingivitis', 'Multiple Sclerosis', 'Tension Headaches', 'Pregnancy', 'Diabetes', 'Anxiety Disorder', 'Hereditary Hemorrhagic Telangiectasia', 'Bronchitis', 'Night Blindness', 'Streptococcal Pharyngitis', 'Dry Eye Syndrome', 'Chronic Migraine', 'Peripheral Artery Disease', 'Typhoid Fever', 'Tendinitis', 'Anorexia', 'Atopic Dermatitis', 'Polycystic Ovary Syndrome', 'Carpal Tunnel Syndrome', 'Malignant Melanoma', 'Gastroesophageal Reflux Disease (GERD)', 'Thyroid Cancer', 'Generalized Anxiety Disorder', "Cushing's Syndrome", 'Diabetes Type 1', 'Liver Disease', 'Es

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

...

100%|██████████| 76/76 [16:55<00:00, 13.36s/it]


Train Loss: 0.0724 | Validation Loss: 0.4559

Epoch 26


100%|██████████| 76/76 [16:52<00:00, 13.32s/it]


Train Loss: 0.0668 | Validation Loss: 0.4542

Early stopping triggered at epoch 26

✅ Classification Report:

                                          precision    recall  f1-score   support

                                    Acne       1.00      1.00      1.00        10
        Age-related Macular Degeneration       0.00      0.00      0.00         1
                       Allergic Rhinitis       1.00      1.00      1.00         1
                         Alopecia Areata       1.00      1.00      1.00         1
                     Alzheimer's Disease       0.00      0.00      0.00         1
                            Appendicitis       0.00      0.00      0.00         1
                               Arthritis       1.00      1.00      1.00        10
                                  Asthma       0.00      0.00      0.00         1
                     Atrial Fibrillation       0.00      0.00      0.00         1
Attention Deficit Hyperactivity Disorder       0.50      1.00      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m